In [3]:
import requests
import pandas as pd

# raw data from: https://string-db.org/cgi/download?sessionId=b6owhyZw1jXy&species_text=Mus+musculus
mouse_ppi = pd.read_csv("mouse_10090.protein.links.v12.0.txt", sep=" ")
mouse_ppi

,protein1,protein2,combined_score
0,10090.ENSMUSP00000000001,10090.ENSMUSP00000027991,889
1,10090.ENSMUSP00000000001,10090.ENSMUSP00000137332,163
2,10090.ENSMUSP00000000001,10090.ENSMUSP00000041756,201
3,10090.ENSMUSP00000000001,10090.ENSMUSP00000075170,969
4,10090.ENSMUSP00000000001,10090.ENSMUSP00000110978,267


In [7]:
mouse_ppi

,protein1,protein2,combined_score
0,10090.ENSMUSP00000000001,10090.ENSMUSP00000027991,889
1,10090.ENSMUSP00000000001,10090.ENSMUSP00000137332,163
2,10090.ENSMUSP00000000001,10090.ENSMUSP00000041756,201
3,10090.ENSMUSP00000000001,10090.ENSMUSP00000075170,969
4,10090.ENSMUSP00000000001,10090.ENSMUSP00000110978,267
...,...,...,...
12684349,10090.ENSMUSP00000159257,10090.ENSMUSP00000047904,197
12684350,10090.ENSMUSP00000159257,10090.ENSMUSP00000062110,367
12684351,10090.ENSMUSP00000159257,10090.ENSMUSP00000099018,169
12684352,10090.ENSMUSP00000159257,10090.ENSMUSP00000023468,252


In [5]:
import numpy as np
ppi_scores = mouse_ppi["combined_score"]
np.min(ppi_scores), np.max(ppi_scores)

(150, 999)

In [6]:
mouse_ppi[mouse_ppi["combined_score"] > 700]

,protein1,protein2,combined_score
0,10090.ENSMUSP00000000001,10090.ENSMUSP00000027991,889
3,10090.ENSMUSP00000000001,10090.ENSMUSP00000075170,969
7,10090.ENSMUSP00000000001,10090.ENSMUSP00000121127,916
8,10090.ENSMUSP00000000001,10090.ENSMUSP00000081569,956
13,10090.ENSMUSP00000000001,10090.ENSMUSP00000025541,760
...,...,...,...
12683968,10090.ENSMUSP00000159241,10090.ENSMUSP00000080242,899
12684079,10090.ENSMUSP00000159241,10090.ENSMUSP00000124205,748
12684116,10090.ENSMUSP00000159241,10090.ENSMUSP00000038137,959
12684164,10090.ENSMUSP00000159241,10090.ENSMUSP00000065819,961


In [ ]:
in_genes = list()
out_genes = list()
in_gene_ids = list()
out_genes_ids = list()
#interaction_score = list()

def uniprot_rest_call(prot_id):
    # Ensembl REST API URL
    url = f"https://rest.uniprot.org/uniprotkb/search?query={prot_id}"
    #https://rest.uniprot.org/uniprotkb/search?query=ENSMUSP00000053551

    # Make GET request to Ensembl API
    response = requests.get(url, headers={"Content-Type": "application/json"})

    # Check if request was successful
    if response.ok:
        # Parse JSON response
        data = response.json()
    gene_name = data["results"][0]["genes"][0]["geneName"]["value"]
    gene_ref = data["results"][0]["uniProtKBCrossReferences"]
    gene_id = None
    for item in gene_ref:
        #print(item)
        if "database" in item and item["database"] == "GeneID":
            gene_id = item["id"]
    return gene_id, gene_name

for i, row in mouse_ppi.iterrows():
    
    in_prot = row["protein1"].split(".")[1]
    out_prot = row["protein2"].split(".")[1]
    #print(in_prot, out_prot)
    in_gene_id, in_gene = uniprot_rest_call(in_prot)
    out_gene_id, out_gene = uniprot_rest_call(out_prot)
    #print(in_gene_id, in_gene, out_gene_id, out_gene)
    in_genes.append(in_gene)
    in_gene_ids.append(in_gene_id)
    
    out_genes.append(out_gene)
    out_genes_ids.append(out_gene_id)
    #print("----")
    if i == 100:
        break
    
mouse_gene_interactions = pd.DataFrame(zip(in_gene_ids, in_genes, out_genes_ids, out_genes), columns=["Gene1_ID", "Gene1_Name", "Gene2_ID", "Gene2_Name"])
mouse_gene_interactions